In [104]:
import pandas as pd

# 작업디렉토리 설정

In [105]:
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp

/data/dhk/physionet.org/files/mimiciv/2.2/hosp


# d_icd_diagnoses파일

In [106]:
df_d_icd_diagnoses = pd.read_csv('d_icd_diagnoses.csv')
df_d_icd_diagnoses.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [107]:
#악성종양 icd_code 
# D00-D09는 암전조상태
df_d_icd_diagnoses_prep = df_d_icd_diagnoses[df_d_icd_diagnoses['icd_code'].str.match(r'^C[0-9]{2}|^D0[0-9]')]
df_d_icd_diagnoses_prep = df_d_icd_diagnoses_prep[['icd_code','icd_version']]
df_d_icd_diagnoses_prep.head()

,icd_code,icd_version
13533,C00,10
13534,C000,10
13535,C001,10
13536,C002,10
13537,C003,10


# diagnoses_icd파일

In [108]:
df_diagnoses_icd = pd.read_csv("diagnoses_icd.csv")
df_diagnoses_icd.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


# 두 dataframe 병합

In [109]:
#암 가진 환자들만 뽑음
df_merge1 = pd.merge(df_diagnoses_icd, df_d_icd_diagnoses_prep, on=['icd_code'])
df_merge1 = df_merge1[df_merge1['seq_num'] == 1]

df_merge1.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y
0,10001401,21544441,1,C675,10,10
3,11282936,22922093,1,C675,10,10
7,13668295,25846146,1,C675,10,10
9,14449673,27244589,1,C675,10,10
13,16455035,29863248,1,C675,10,10


# 암 가진 환자들의 다른 기저질환

In [110]:
# #병합된 df에서 subject_icd 추출
# masked_subject_id = df_merge1['subject_id'].values


In [111]:
# under_condition =  df_diagnoses_icd[df_diagnoses_icd['subject_id'].isin(masked_subject_id)][['subject_id','icd_code']]
# under_condition

# patients파일

In [112]:
df_patients = pd.read_csv("patients.csv")
df_patients.head(3)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN


# merge1과 patients 병합

In [113]:
merge2 = pd.merge(df_merge1,df_patients, on=['subject_id'])
merge2.head()
# len(merge2)
#이코드가 암환자 정보

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10001401,21544441,1,C675,10,10,F,89,2131,2014 - 2016,NaN
1,11282936,22922093,1,C675,10,10,M,73,2193,2014 - 2016,NaN
2,13668295,25846146,1,C675,10,10,F,88,2177,2014 - 2016,NaN
3,14449673,27244589,1,C675,10,10,M,73,2130,2017 - 2019,NaN
4,16455035,29863248,1,C675,10,10,M,75,2129,2014 - 2016,NaN


# 환자나이 계산하기 위해 merge

In [114]:
df_admission = pd.read_csv("admissions.csv")
merge2_admission = pd.merge(merge2,df_admission, on=['subject_id','hadm_id'])
merge2_admission

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,gender,anchor_age,anchor_year,anchor_year_group,...,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10001401,21544441,1,C675,10,10,F,89,2131,2014 - 2016,...,P872K3,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
1,11282936,22922093,1,C675,10,10,M,73,2193,2014 - 2016,...,P872K3,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
2,13668295,25846146,1,C675,10,10,F,88,2177,2014 - 2016,...,P857XC,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,WIDOWED,WHITE,NaN,NaN,0
3,14449673,27244589,1,C675,10,10,M,73,2130,2017 - 2019,...,P857XC,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
4,16455035,29863248,1,C675,10,10,M,75,2129,2014 - 2016,...,P980B9,PHYSICIAN REFERRAL,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7997,18274431,27878903,1,C172,10,10,M,82,2183,2011 - 2013,...,P3417E,AMBULATORY SURGERY TRANSFER,REHAB,Medicare,ENGLISH,SINGLE,WHITE,2187-12-09 06:19:00,2187-12-09 14:06:00,0
7998,18942246,25534797,1,C031,10,10,M,58,2146,2017 - 2019,...,P80GYA,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2146-02-13 20:58:00,2146-02-14 09:05:00,0
7999,19159075,22141379,1,C44799,10,10,M,91,2173,2017 - 2019,...,P62A9I,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,NaN,NaN,0
8000,19282415,28695316,1,C004,10,10,M,67,2184,2017 - 2019,...,P52DD2,PHYSICIAN REFERRAL,CHRONIC/LONG TERM ACUTE CARE,Medicare,?,NaN,WHITE,NaN,NaN,0


# 입원시점에서 환자나이 계산

In [115]:
# 출생 연도 계산
print(merge2_admission[['anchor_year','anchor_age']].head(5))
print("~~")

##adgmit_age가 이상한 경우
#  환자의 나이가 89세 이상인 경우, 데이터 보호를 위해 모든 연령이 89세로 고정되어 있다는 것입니다.
merge2_admission = merge2_admission[merge2_admission['anchor_age']<89]

# admittime 열을 datetime 형식으로 변환
merge2_admission['birth_year'] = merge2_admission['anchor_year'] - merge2_admission['anchor_age']

# 입원 연도 추출
admit_year = pd.to_datetime(merge2_admission['admittime']).dt.year

# 입원 시점의 나이 계산
merge2_admission['admit_age'] = admit_year - merge2_admission['birth_year']

#인덱스 초기화
merge2_admission.reset_index(drop=True, inplace=True)

#결과확인
print(merge2_admission[['subject_id', 'admittime', 'admit_age','birth_year','anchor_year','anchor_age']].head())

   anchor_year  anchor_age
0         2131          89
1         2193          73
2         2177          88
3         2130          73
4         2129          75
~~
   subject_id            admittime  admit_age  birth_year  anchor_year  \
0    11282936  2193-05-08 00:00:00         73        2120         2193   
1    13668295  2177-05-15 00:00:00         88        2089         2177   
2    14449673  2130-10-17 00:00:00         73        2057         2130   
3    16455035  2129-11-03 14:00:00         75        2054         2129   
4    17410051  2124-10-08 20:10:00         86        2038         2124   

   anchor_age  
0          73  
1          88  
2          73  
3          75  
4          86  


/tmp/ipykernel_37756/4117411835.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge2_admission['birth_year'] = merge2_admission['anchor_year'] - merge2_admission['anchor_age']
/tmp/ipykernel_37756/4117411835.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge2_admission['admit_age'] = admit_year - merge2_admission['birth_year']


# 예측했을때 써야할 피쳐들만 가지고왔는데 의논이 필요한듯

In [116]:
merge2_admission = merge2_admission[['subject_id','hadm_id','admit_age','seq_num','gender','dod','deathtime','insurance','language','marital_status','race','hospital_expire_flag','birth_year',]]
merge2_admission

,subject_id,hadm_id,admit_age,seq_num,gender,dod,deathtime,insurance,language,marital_status,race,hospital_expire_flag,birth_year
0,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120
1,13668295,25846146,88,1,F,NaN,NaN,Medicare,ENGLISH,WIDOWED,WHITE,0,2089
2,14449673,27244589,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2057
3,16455035,29863248,75,1,M,NaN,NaN,Medicare,ENGLISH,MARRIED,WHITE,0,2054
4,17410051,24586421,86,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7871,18252838,22671212,59,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2072
7872,18274431,27878903,86,1,M,NaN,NaN,Medicare,ENGLISH,SINGLE,WHITE,0,2101
7873,18942246,25534797,58,1,M,2146-04-19,NaN,Other,ENGLISH,MARRIED,WHITE,0,2088
7874,19282415,28695316,67,1,M,NaN,NaN,Medicare,?,NaN,WHITE,0,2117


# death비율과 death하지 않은비율

In [117]:
merge2_admission['hospital_expire_flag'].value_counts()

hospital_expire_flag
0    7548
1     328
Name: count, dtype: int64

# death 종양환자 평균나이 비교

In [118]:
print(f"종양환자의 평균나이 :{ merge2_admission['admit_age'].mean() }")

C_death = merge2_admission[merge2_admission['hospital_expire_flag']==1]
C_death_mean_age = C_death['admit_age'].mean()
print(f"death한 종양환자의 평균나이 :{C_death_mean_age}")

C_not_death = merge2_admission[merge2_admission['hospital_expire_flag']== 0]
C_not_death_mean_age = C_not_death['admit_age'].mean()
print(f"death하지않은 종양환자의 평균나이 :{C_not_death_mean_age}")

종양환자의 평균나이 :64.21800406297614
death한 종양환자의 평균나이 :66.50914634146342
death하지않은 종양환자의 평균나이 :64.11844197138315


# omr(환자 몸무게 혈압 등등) merge

In [119]:
omr = pd.read_csv("omr.csv")
# omr
merge3_omr = pd.merge(merge2_admission,omr, on=['subject_id'])
merge3_omr

,subject_id,hadm_id,admit_age,seq_num_x,gender,dod,deathtime,insurance,language,marital_status,race,hospital_expire_flag,birth_year,chartdate,seq_num_y,result_name,result_value
0,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2192-11-26,1,Blood Pressure,150/80
1,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-01-22,1,Blood Pressure,150/80
2,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-02-19,1,Blood Pressure,154/92
3,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-02-19,1,BMI (kg/m2),24.5
4,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-02-19,1,Height (Inches),71.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985909,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-07-21,1,Weight (Lbs),151.8
985910,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-08-18,1,Blood Pressure,124/56
985911,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-08-18,1,BMI (kg/m2),26.9
985912,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-08-18,1,Height (Inches),64


# pharmacy(환자 약물 처방 및 투약 내역) merge

In [120]:
pharmacy = pd.read_csv("pharmacy.csv")
pharmacy = pharmacy[['subject_id', 'hadm_id','poe_id','medication','frequency','doses_per_24_hrs']]
# pharmacy
merge4_pharmacy = pd.merge(merge3_omr,pharmacy, on=['subject_id','hadm_id'])
merge4_pharmacy

/tmp/ipykernel_37756/2794743079.py:1: DtypeWarning: Columns (16,18,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  pharmacy = pd.read_csv("pharmacy.csv")


,subject_id,hadm_id,admit_age,seq_num_x,gender,dod,deathtime,insurance,language,marital_status,...,hospital_expire_flag,birth_year,chartdate,seq_num_y,result_name,result_value,poe_id,medication,frequency,doses_per_24_hrs
0,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-67,Labetalol,ONCE,1.0
1,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-191,Acetaminophen,TID,3.0
2,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-215,NaN,ASDIR,NaN
3,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-111,Metoprolol Tartrate,Q4H:PRN,NaN
4,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-50,HYDROmorphone (Dilaudid),Q5MIN:PRN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56003648,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-328,NaN,ASDIR,NaN
56003649,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-621,PredniSONE,DAILY,1.0
56003650,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-419,Metoprolol Tartrate,ONCE MR1,1.0
56003651,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-726,Haloperidol,ONCE MR1,1.0


# prescriptions(약물 처방 기록) merge

In [126]:
prescriptions = pd.read_csv("prescriptions.csv")
prescriptions = prescriptions[['subject_id', 'hadm_id','poe_id','drug','dose_val_rx','form_unit_disp','doses_per_24_hrs']]
merge5_prescriptions = pd.merge(merge4_pharmacy,prescriptions, on=['subject_id','hadm_id','poe_id'])
merge5_prescriptions

/tmp/ipykernel_37756/1010516667.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv("prescriptions.csv")


,subject_id,hadm_id,admit_age,seq_num_x,gender,dod,deathtime,insurance,language,marital_status,...,result_name,result_value,poe_id,medication,frequency,doses_per_24_hrs_x,drug,dose_val_rx,form_unit_disp,doses_per_24_hrs_y
0,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,150/80,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
1,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,150/80,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
2,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,154/92,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
3,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,BMI (kg/m2),24.5,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
4,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Height (Inches),71.5,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97646596,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,Weight (Lbs),151.8,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
97646597,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,124/56,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
97646598,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,BMI (kg/m2),26.9,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
97646599,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,Height (Inches),64,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
